In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.stats as sm_stats

import datetime
import re
from sklearn.impute import SimpleImputer, KNNImputer
from numpy import percentile

import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectPercentile, SelectFromModel
from sklearn.feature_selection import mutual_info_regression, chi2, f_regression, f_classif
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from env import *

In [2]:
pd.set_option('display.max_columns', None)

# Loading

In [3]:
df = pd.read_json(CLEANED_DATASET_JSON_INTERMEDIATE, orient ='records')

In [4]:
df[:5]

order_x     status     started at  corrected start at   device       os  \
0        8  completed  1648543810000       1648551010000  desktop  Windows   
1       13  completed  1648546592000       1648553792000  desktop     OS X   
2       17  completed  1648555295000       1648562495000  desktop  Windows   
3       18  completed  1648557191000       1648564391000  desktop   Ubuntu   
4       22  completed  1648575332000       1648582532000  desktop  Windows   

   browser location time taken                    Session  Task  \
0   Chrome       SK   00:13:58  gsd4FIhsnwaOIMOtoSqX4geVy     1   
1   Chrome       SK   00:13:04  IKW3ZZ9lcX5Z3vSKEghI73tuI     1   
2   Chrome       SK   00:06:54  Nhkvw5dbEwImQaB1qZNPqm7Qh     2   
3  Firefox       SK   00:07:19  5HPhIkC2ryz2x5IiTSDGramvs     2   
4   Chrome       SK   00:06:17  RBPuYafOHOIdMXLQ07pBerkXl     2   

   Limesurvey_Seed  order_y  indicator_fg  indicator_h  id  \
0       1445944018        1             1            0   6   
1       1313525205        3             1            0   8   
2        188959783        2             0            1  23   
3       2103452617        3             0            1  24   
4       1707296003        5             0            1  26   

            submitdate  lastpage startlanguage  kick        seed  \
0  2022-03-29 11:03:59        66            sk     0  1445944018   
1  2022-03-29 11:49:25        66            sk     0  1313525205   
2  2022-03-29 14:08:20        66            sk     0   188959783   
3  2022-03-29 14:40:25        66            sk     0  2103452617   
4  2022-03-29 19:41:44        66            sk     0  1707296003   

             startdate            datestamp  total_time  mean_time  \
0  2022-03-29 10:50:48  2022-03-29 11:03:59      253.26   5.276250   
1  2022-03-29 11:40:05  2022-03-29 11:49:25      325.25   6.776042   
2  2022-03-29 14:02:04  2022-03-29 14:08:20      227.94   4.748750   
3  2022-03-29 14:33:26  2022-03-29 14:40:25      272.48   5.676667   
4  2022-03-29 19:36:13  2022-03-29 19:41:44      223.86   4.663750   

   total_points  mean_points  mean_points_check  mean_bfi2_e  mean_bfi2_a  \
0           184     3.833333           3.833333     2.916667     4.416667   
1           148     3.083333           3.083333     2.000000     3.833333   
2           175     3.645833           3.645833     3.166667     4.333333   
3           159     3.312500           3.312500     2.500000     3.750000   
4           122     2.541667           2.541667     1.416667     4.666667   

   mean_bfi2_c  mean_bfi2_n  mean_bfi2_e_sociability  \
0     4.166667     3.833333                     2.25   
1     3.750000     2.750000                     1.50   
2     4.083333     3.000000                     2.75   
3     3.416667     3.583333                     1.50   
4     3.083333     1.000000                     1.00   

   mean_bfi2_e_assertiveness  mean_bfi2_e_energy_level  \
0                       2.75                      3.75   
1                       2.00                      2.50   
2                       3.00                      3.75   
3                       3.00                      3.00   
4                       1.00                      2.25   

   mean_bfi2_a_compassion  mean_bfi2_a_respectfulness  mean_bfi2_a_trust  \
0                    4.75                        4.50               4.00   
1                    4.25                        4.75               2.50   
2                    4.50                        4.25               4.25   
3                    3.25                        4.25               3.75   
4                    5.00                        5.00               4.00   

   mean_bfi2_c_organization  mean_bfi2_c_productiveness  \
0                      4.25                        4.00   
1                      3.75                        3.25   
2                      3.75                        3.75   
3                      4.00                        3.25   
4                      5.00

In [5]:
len(df)

112

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Columns: 1581 entries, order_x to Page_67-react_time
dtypes: bool(68), float64(876), int64(613), object(24)
memory usage: 1.3+ MB


# Preprocessing

In [7]:
df[:5]

order_x     status     started at  corrected start at   device       os  \
0        8  completed  1648543810000       1648551010000  desktop  Windows   
1       13  completed  1648546592000       1648553792000  desktop     OS X   
2       17  completed  1648555295000       1648562495000  desktop  Windows   
3       18  completed  1648557191000       1648564391000  desktop   Ubuntu   
4       22  completed  1648575332000       1648582532000  desktop  Windows   

   browser location time taken                    Session  Task  \
0   Chrome       SK   00:13:58  gsd4FIhsnwaOIMOtoSqX4geVy     1   
1   Chrome       SK   00:13:04  IKW3ZZ9lcX5Z3vSKEghI73tuI     1   
2   Chrome       SK   00:06:54  Nhkvw5dbEwImQaB1qZNPqm7Qh     2   
3  Firefox       SK   00:07:19  5HPhIkC2ryz2x5IiTSDGramvs     2   
4   Chrome       SK   00:06:17  RBPuYafOHOIdMXLQ07pBerkXl     2   

   Limesurvey_Seed  order_y  indicator_fg  indicator_h  id  \
0       1445944018        1             1            0   6   
1       1313525205        3             1            0   8   
2        188959783        2             0            1  23   
3       2103452617        3             0            1  24   
4       1707296003        5             0            1  26   

            submitdate  lastpage startlanguage  kick        seed  \
0  2022-03-29 11:03:59        66            sk     0  1445944018   
1  2022-03-29 11:49:25        66            sk     0  1313525205   
2  2022-03-29 14:08:20        66            sk     0   188959783   
3  2022-03-29 14:40:25        66            sk     0  2103452617   
4  2022-03-29 19:41:44        66            sk     0  1707296003   

             startdate            datestamp  total_time  mean_time  \
0  2022-03-29 10:50:48  2022-03-29 11:03:59      253.26   5.276250   
1  2022-03-29 11:40:05  2022-03-29 11:49:25      325.25   6.776042   
2  2022-03-29 14:02:04  2022-03-29 14:08:20      227.94   4.748750   
3  2022-03-29 14:33:26  2022-03-29 14:40:25      272.48   5.676667   
4  2022-03-29 19:36:13  2022-03-29 19:41:44      223.86   4.663750   

   total_points  mean_points  mean_points_check  mean_bfi2_e  mean_bfi2_a  \
0           184     3.833333           3.833333     2.916667     4.416667   
1           148     3.083333           3.083333     2.000000     3.833333   
2           175     3.645833           3.645833     3.166667     4.333333   
3           159     3.312500           3.312500     2.500000     3.750000   
4           122     2.541667           2.541667     1.416667     4.666667   

   mean_bfi2_c  mean_bfi2_n  mean_bfi2_e_sociability  \
0     4.166667     3.833333                     2.25   
1     3.750000     2.750000                     1.50   
2     4.083333     3.000000                     2.75   
3     3.416667     3.583333                     1.50   
4     3.083333     1.000000                     1.00   

   mean_bfi2_e_assertiveness  mean_bfi2_e_energy_level  \
0                       2.75                      3.75   
1                       2.00                      2.50   
2                       3.00                      3.75   
3                       3.00                      3.00   
4                       1.00                      2.25   

   mean_bfi2_a_compassion  mean_bfi2_a_respectfulness  mean_bfi2_a_trust  \
0                    4.75                        4.50               4.00   
1                    4.25                        4.75               2.50   
2                    4.50                        4.25               4.25   
3                    3.25                        4.25               3.75   
4                    5.00                        5.00               4.00   

   mean_bfi2_c_organization  mean_bfi2_c_productiveness  \
0                      4.25                        4.00   
1                      3.75                        3.25   
2                      3.75                        3.75   
3                      4.00                        3.25   
4                      5.00

## Generate column names

In [8]:
reversed_questions = [11, 16, 26, 31, 36, 51, 12, 17, 22, 37, 42, 47, 3, 8, 23, 28, 48, 58, 4, 9, 24, 29, 44, 49, 5, 25, 30, 45, 50, 55]
reversed_questions.sort()
limesurvey_columns = ["bfi{}".format(str(i)) if i not in reversed_questions else "rbfi{}".format(str(i)) for i in range(1, 61)]

k = 5
deleted = limesurvey_columns[k-1::k]
del limesurvey_columns[k-1::k]
# print(limesurvey_columns)

demographic_columns = ["G03Q{:02d}".format(i) for i in range(1, 15)]
print(demographic_columns)

['G03Q01', 'G03Q02', 'G03Q03', 'G03Q04', 'G03Q05', 'G03Q06', 'G03Q07', 'G03Q08', 'G03Q09', 'G03Q10', 'G03Q11', 'G03Q12', 'G03Q13', 'G03Q14']


In [9]:
print(limesurvey_columns)

['bfi1', 'bfi2', 'rbfi3', 'rbfi4', 'bfi6', 'bfi7', 'rbfi8', 'rbfi9', 'rbfi11', 'rbfi12', 'bfi13', 'bfi14', 'rbfi16', 'rbfi17', 'bfi18', 'bfi19', 'bfi21', 'rbfi22', 'rbfi23', 'rbfi24', 'rbfi26', 'bfi27', 'rbfi28', 'rbfi29', 'rbfi31', 'bfi32', 'bfi33', 'bfi34', 'rbfi36', 'rbfi37', 'bfi38', 'bfi39', 'bfi41', 'rbfi42', 'bfi43', 'rbfi44', 'bfi46', 'rbfi47', 'rbfi48', 'rbfi49', 'rbfi51', 'bfi52', 'bfi53', 'bfi54', 'bfi56', 'bfi57', 'rbfi58', 'bfi59']


In [10]:
metric_names = ['time', 'duration', 'distance', 'x_axis_distance', 'y_axis_distance', 'real_ideal_trajectory_diff', 'max_deviation', 'velocity', 
                'x_axis_vel', 'y_axis_vel', 'acc', 'x_axis_acc', 'y_axis_acc', 'auc_diff', 'clicks', 'x_flips', 'y_flips', 'visits', 'scrolling', 
                'init_time', 'react_time']   

metric_columns = []
for x in range(68):
    for name in metric_names:
        metric_columns.append('Page_{}-{}'.format(x, name))
# metric_columns

In [11]:
all_columns = ['Session', 'indicator_fg'] + limesurvey_columns + demographic_columns + metric_columns
len(all_columns)

1492

In [12]:
df[all_columns]

Session  indicator_fg  bfi1  bfi2  rbfi3  rbfi4  bfi6  \
0    gsd4FIhsnwaOIMOtoSqX4geVy             1     4     5      5      3     4   
1    IKW3ZZ9lcX5Z3vSKEghI73tuI             1     2     4      2      4     3   
2    Nhkvw5dbEwImQaB1qZNPqm7Qh             0     4     5      3      3     3   
3    5HPhIkC2ryz2x5IiTSDGramvs             0     3     4      5      4     2   
4    RBPuYafOHOIdMXLQ07pBerkXl             0     1     5      5      1     1   
..                         ...           ...   ...   ...    ...    ...   ...   
107  GIszPEY36M0Y6Zaq2mJQYE2Ev             1     5     5      5      5     5   
108  MHWlCKW1Rth7IufdKQ53gcHoW             1     4     3      2      4     2   
109  yQOKd4r41wCPrKd5cVFaAuJED             0     4     5      4      4     3   
110  QJpzM76mOzzx41jlZ2cD68ZVr             1     5     5      4      4     5   
111  DPESrzxKVWTAlVhksRGpPRAKy             1     4     4      4      4     5   

     bfi7  rbfi8  rbfi9  rbfi11  rbfi12  bfi13  bfi14  rbfi16  rbfi17  bfi18  \
0       5      4      3       5       4      4      5       1       5      3   
1       5      1      2       2       2      4      1       1       5      4   
2       4      3      3       4       5      5      4       3       4      4   
3       4      2      3       3       4      4      4       1       4      4   
4       5      1      1       4       5      3      1       1       5      5   
..    ...    ...    ...     ...     ...    ...    ...     ...     ...    ...   
107     5      5      5       5       5      5      5       2       5      5   
108     5      2      5       4       2      5      4       2       5      4   
109     5      2      2       1       3      5      5       2       5      4   
110     5      5      4       4       4      5      5       3       4      5   
111     3      3      4       4       4      4      4       5       2      4   

     bfi19  bfi21  rbfi22  rbfi23  rbfi24  rbfi26  bfi27  rbfi28  rbfi29  \
0        4      1       5       4       4       4      5       4       3   
1        2      1       5       2       2       2      2       4       2   
2        3      4       4       4       2       3      4       4       2   
3        2      3       5       4       4       4      4       2       4   
4        1      1       5       1       1       1      5       1       1   
..     ...    ...     ...     ...     ...     ...    ...     ...     ...   
107      1      4       4       5       4       4      4       4       5   
108      3      2       5       4       4       3      4       4       5   
109      2      3       5       3       1       1      5       4       4   
110      5      4       4       4       5       4      4       4       5   
111      5      4       4       5       4       3      4       4       5   

     rbfi31  bfi32  bfi33  bfi34  rbfi36  rbfi37  bfi38  bfi39  bfi41  rbfi42  \
0         1      4      4      3       2       3      4      4      3       4   
1         1      4      4      4       2       4      5      4      4       3   
2         1      5      4      3       2       5      4      3      4       4   
3         1      3      4      5       3       4      4      2      3       4   
4         1      5      5      1       1       5      3      1      1       2   
..      ...    ...    ...    ...     ...     ...    ...    ...    ...     ...   
107       4      5      4      4       3       5      5      5      4       4   
108       2      3      3      4       3       5      4      2      4       3   
109       2      5      3      3       1       5      4      1      1       2   
110       4      5      5      4       3       5      5      4      5       2   
111       2      4      5      4       3       4      5      4      5       5   

     bfi43  rbfi44  bfi46  rbfi47  rbfi48  rbfi49  rbfi51  bfi52  bfi53  \
0        4       5      3       5       5       3       4      5      4   
1        5       4      2       4       5       2       2      5 

## Transformation of **1 row = 1 session** dataframe to **1 row = 1 pageview** dataframe

In [13]:
new_columns_names = ['Session', 'indicator_fg', 'Page', 'Likert_value', 'Big5'] + metric_names
df_new = pd.DataFrame(columns = new_columns_names)
df_new

,Session,indicator_fg,Page,Likert_value,Big5,time,duration,distance,x_axis_distance,y_axis_distance,real_ideal_trajectory_diff,max_deviation,velocity,x_axis_vel,y_axis_vel,acc,x_axis_acc,y_axis_acc,auc_diff,clicks,x_flips,y_flips,visits,scrolling,init_time,react_time


Cycle iterating over dataframe to create new dataframe, more suitable for machine learning.

In [14]:
big5_pages = range(3, 51)
dem_pages = range(52, 66)

for index, row in df.iterrows():
    if index % 10 == 0:
        print(index)
    # if index == 5:
    #     break
    limesurvey_counter = 0
    demography_counter = 0
    for x in range(68):
        new_row = {'Session': row['Session'], 'indicator_fg': row['indicator_fg'], 'Page': x}
        if x in big5_pages:
            new_row['Likert_value'] = row[limesurvey_columns[limesurvey_counter]]
            new_row['Big5'] = 1
            limesurvey_counter += 1
        elif x in dem_pages:
            new_row['Likert_value'] = row[demographic_columns[demography_counter]]
            new_row['Big5'] = 0
            demography_counter += 1
        else:
            continue
            
        for name in metric_names:
            new_row[name] = row['Page_{}-{}'.format(x, name)]
        df_new = df_new.append(new_row, ignore_index=True)

0
10
20
30
40
50
60
70
80
90
100
110


In [15]:
len(df_new)

6944

In [16]:
df_new.columns

Index(['Session', 'indicator_fg', 'Page', 'Likert_value', 'Big5', 'time',
       'duration', 'distance', 'x_axis_distance', 'y_axis_distance',
       'real_ideal_trajectory_diff', 'max_deviation', 'velocity', 'x_axis_vel',
       'y_axis_vel', 'acc', 'x_axis_acc', 'y_axis_acc', 'auc_diff', 'clicks',
       'x_flips', 'y_flips', 'visits', 'scrolling', 'init_time', 'react_time'],
      dtype='object')

In [17]:
len(df_new.columns)

26

In [18]:
df_new[20:65]

,Session,indicator_fg,Page,Likert_value,Big5,time,duration,distance,x_axis_distance,y_axis_distance,real_ideal_trajectory_diff,max_deviation,velocity,x_axis_vel,y_axis_vel,acc,x_axis_acc,y_axis_acc,auc_diff,clicks,x_flips,y_flips,visits,scrolling,init_time,react_time
20,gsd4FIhsnwaOIMOtoSqX4geVy,1,23,4,1,816,3439,585.51655,467.15625,293.0,33.331936,43.875465,0.717545,0.572495,0.359069,0.000879,0.000702,0.00044,125908.40625,2,1,1,1,False,2252,2697
21,gsd4FIhsnwaOIMOtoSqX4geVy,1,24,5,1,3967,5036,862.311496,774.46875,275.0,100.246322,82.055128,0.217371,0.195228,0.069322,0.000055,0.000049,0.000017,177008.90625,2,2,1,1,False,708,4418
22,gsd4FIhsnwaOIMOtoSqX4geVy,1,25,4,1,2249,3359,521.033699,379.5,294.0,40.671436,41.100637,0.231673,0.168742,0.130725,0.000103,0.000075,0.000058,91884.890625,2,1,1,1,False,555,2571
23,gsd4FIhsnwaOIMOtoSqX4geVy,1,26,3,1,1287,5793,447.687184,219.65625,295.0,144.678753,14.569639,0.347853,0.170673,0.229215,0.00027,0.000133,0.000178,90454.03125,2,2,1,1,False,4493,5129
24,gsd4FIhsnwaOIMOtoSqX4geVy,1,27,1,1,1750,4351,960.143194,866.25,337.0,34.810899,26.581565,0.548653,0.495,0.192571,0.000314,0.000283,0.00011,188641.40625,2,1,1,1,False,2257,3543
25,gsd4FIhsnwaOIMOtoSqX4geVy,1,28,4,1,3261,3952,747.890023,644.53125,336.0,31.215215,45.520411,0.229344,0.197648,0.103036,0.00007,0.000061,0.000032,86667.796875,2,1,3,1,False,204,3259
26,gsd4FIhsnwaOIMOtoSqX4geVy,1,29,4,1,1758,3476,1088.706483,943.59375,323.0,629.777074,126.615799,0.619287,0.536743,0.183732,0.000352,0.000305,0.000105,167027.4375,2,4,5,1,False,1710,2779
27,gsd4FIhsnwaOIMOtoSqX4geVy,1,30,3,1,2094,3625,849.732147,672.375,337.0,445.836243,175.280288,0.405794,0.321096,0.160936,0.000194,0.000153,0.000077,72289.59375,3,2,4,1,False,1531,2937
28,gsd4FIhsnwaOIMOtoSqX4geVy,1,31,2,1,1099,5745,755.88037,693.0,279.0,4.397713,11.670276,0.687789,0.630573,0.253867,0.000626,0.000574,0.000231,126787.546875,2,1,1,1,False,4134,5020
29,gsd4FIhsnwaOIMOtoSqX4geVy,1,32,3,1,5388,5963,626.677521,392.90625,303.0,229.455196,116.700173,0.11631,0.072922,0.056236,0.000022,0.000014,0.00001,4028.0625,2,2,2,1,False,150,5158


# Export dataset

In [19]:
result = df_new.to_json(orient="records")
with open(DATASET_ONE_PV_PER_ONE_ROW, 'w') as outfile:
    outfile.write(result)

# Dummy selection of Attributes

Code for this whole section is taken from my project developed on the subject Intelligent Data Analysis 2021/2022.

In [20]:
columns_metrics = BIG5_MEAN_COLS + BIG5_SUM_COLS + DEM_MEAN_COLS + DEM_SUM_COLS
columns_metrics.remove('big5_sum_scrolling')
columns_metrics.remove('dem_sum_scrolling')

all_metrics = COLUMNS_STATIC + columns_metrics

In [21]:
columns_metrics_ind = BIG5_MEAN_COLS + BIG5_SUM_COLS + COLUMNS_STATIC + ['indicator_fg']
columns_metrics_ind.remove('big5_sum_scrolling')

In [22]:
len(columns_metrics_ind)

40

In [23]:
# whole dataset
# df_nums = df[columns_metrics_ind] 

In [24]:
# one page per row dataset
df_nums = df_new[df_new['Big5'] == 1]
df_nums = df_nums.drop(["Session"], axis=1)
df_nums.dropna(inplace=True)
df_nums

,indicator_fg,Page,Likert_value,Big5,time,duration,distance,x_axis_distance,y_axis_distance,real_ideal_trajectory_diff,max_deviation,velocity,x_axis_vel,y_axis_vel,acc,x_axis_acc,y_axis_acc,auc_diff,clicks,x_flips,y_flips,visits,scrolling,init_time,react_time
0,1,3,4,1,14236,14893,4099.440429,3635.15625,1076.0,3434.874661,539.568132,0.287963,0.25535,0.075583,0.00002,0.000018,0.000005,50776.6875,2,8,8,1,False,261,7861
1,1,4,5,1,2448,4529,1298.103346,1235.4375,354.0,456.372879,123.64276,0.530271,0.504672,0.144608,0.000217,0.000206,0.000059,161737.125,2,2,1,1,False,470,2520
2,1,5,5,1,4051,4355,976.613565,885.84375,329.0,241.107925,18.563896,0.24108,0.218673,0.081215,0.00006,0.000054,0.00002,74979.609375,2,2,4,1,False,281,3316
3,1,6,3,1,1432,6039,478.125316,256.78125,300.0,65.496732,81.348974,0.333886,0.179317,0.209497,0.000233,0.000125,0.000146,18163.40625,2,2,3,1,False,2264,3262
4,1,7,4,1,3648,4513,645.698909,551.71875,306.0,177.745514,71.096966,0.177001,0.151239,0.083882,0.000049,0.000041,0.000023,91599.75,2,2,7,1,False,417,3670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6925,1,46,4,1,5351,7969,1219.932904,1080.234375,372.5,545.641375,224.092544,0.227982,0.201875,0.069613,0.000043,0.000038,0.000013,82819.848633,2,2,1,1,False,2174,7044
6926,1,47,4,1,4435,5323,977.258222,861.09375,357.5,311.645115,84.937991,0.220351,0.194159,0.080609,0.00005,0.000044,0.000018,126346.655273,2,1,3,1,False,539,4414
6927,1,48,4,1,3468,4208,972.407191,782.460938,447.5,163.306492,107.518036,0.280394,0.225623,0.129037,0.000081,0.000065,0.000037,127453.637695,2,1,6,1,False,336,3363
6928,1,49,4,1,9533,12086,1526.058989,1223.320312,758.75,881.519388,282.290524,0.160082,0.128325,0.079592,0.000017,0.000013,0.000008,60848.583984,3,2,10,1,False,2201,11299


In [25]:
X = df_nums.drop(["indicator_fg"], axis=1)
y = df_nums["indicator_fg"]
y=y.astype('int')

In [26]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
colsVT = sel.fit_transform(X)
colsVT[0].size

15

# Simple feature selection for testing different approaches

In [27]:
def orderColumns(X, y, typeOfSelection):
    selector = SelectKBest(typeOfSelection, k ='all')
    X_selected = selector.fit_transform(X, y)
    scores = selector.scores_
    column_names = X.columns[selector.get_support()]
    indices = [x for _,x in sorted(zip(scores,column_names), reverse=True)]
    
    return indices

In [28]:
MI_column_order = orderColumns(X, y, mutual_info_regression)
print(MI_column_order)

['Likert_value', 'y_flips', 'Big5', 'distance', 'x_axis_acc', 'x_axis_vel', 'y_axis_distance', 'auc_diff', 'scrolling', 'visits', 'init_time', 'velocity', 'x_axis_distance', 'max_deviation', 'real_ideal_trajectory_diff', 'react_time', 'y_axis_acc', 'y_axis_vel', 'x_flips', 'time', 'duration', 'clicks', 'acc', 'Page']


In [29]:
Chi2_column_order = orderColumns(X, y, chi2)
print(Chi2_column_order)

['auc_diff', 'time', 'x_axis_distance', 'distance', 'max_deviation', 'real_ideal_trajectory_diff', 'Likert_value', 'duration', 'y_axis_distance', 'y_flips', 'x_axis_vel', 'velocity', 'clicks', 'x_flips', 'visits', 'y_axis_vel', 'Page', 'x_axis_acc', 'acc', 'y_axis_acc', 'Big5', 'scrolling', 'init_time', 'react_time']


In [30]:
Fv_column_order = orderColumns(X, y, f_regression)
print(Fv_column_order)

['Likert_value', 'Big5', 'x_axis_vel', 'auc_diff', 'velocity', 'max_deviation', 'x_axis_acc', 'visits', 'acc', 'y_flips', 'clicks', 'y_axis_vel', 'time', 'x_axis_distance', 'distance', 'y_axis_acc', 'real_ideal_trajectory_diff', 'x_flips', 'y_axis_distance', 'duration', 'Page', 'scrolling', 'init_time', 'react_time']


C:\Users\PeterSmrecek\Documents\BP\BP-venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\PeterSmrecek\Documents\BP\BP-venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\PeterSmrecek\Documents\BP\BP-venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:358: RuntimeWarning: invalid value encountered in true_divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


In [31]:
def orderColumnsPrc(X, y, estimator):
    selector = SelectPercentile(estimator, percentile=100)
    X_selected = selector.fit_transform(X, y)
    scores = selector.scores_
    column_names = X.columns[selector.get_support()]
    indices = [x for _,x in sorted(zip(scores,column_names), reverse=True)]
    
    return indices

In [32]:
PrcCH_column_order = orderColumnsPrc(X, y, chi2)
print(PrcCH_column_order)

['auc_diff', 'time', 'x_axis_distance', 'distance', 'max_deviation', 'real_ideal_trajectory_diff', 'Likert_value', 'duration', 'y_axis_distance', 'y_flips', 'x_axis_vel', 'velocity', 'clicks', 'x_flips', 'visits', 'y_axis_vel', 'Page', 'x_axis_acc', 'acc', 'y_axis_acc', 'Big5', 'scrolling', 'init_time', 'react_time']


In [33]:
PrcFCL_column_order = orderColumnsPrc(X, y, f_classif)
print(PrcFCL_column_order)

['Likert_value', 'Big5', 'x_axis_vel', 'auc_diff', 'velocity', 'max_deviation', 'x_axis_acc', 'visits', 'acc', 'y_flips', 'clicks', 'y_axis_vel', 'time', 'x_axis_distance', 'distance', 'y_axis_acc', 'real_ideal_trajectory_diff', 'x_flips', 'y_axis_distance', 'duration', 'Page', 'scrolling', 'init_time', 'react_time']


C:\Users\PeterSmrecek\Documents\BP\BP-venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 2 21] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\PeterSmrecek\Documents\BP\BP-venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [34]:
PrcFRE_column_order = orderColumnsPrc(X, y, f_regression)
print(PrcFRE_column_order)

['Likert_value', 'Big5', 'x_axis_vel', 'auc_diff', 'velocity', 'max_deviation', 'x_axis_acc', 'visits', 'acc', 'y_flips', 'clicks', 'y_axis_vel', 'time', 'x_axis_distance', 'distance', 'y_axis_acc', 'real_ideal_trajectory_diff', 'x_flips', 'y_axis_distance', 'duration', 'Page', 'scrolling', 'init_time', 'react_time']


C:\Users\PeterSmrecek\Documents\BP\BP-venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\PeterSmrecek\Documents\BP\BP-venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\PeterSmrecek\Documents\BP\BP-venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:358: RuntimeWarning: invalid value encountered in true_divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


In [35]:
selectorSFM = SelectFromModel(RandomForestClassifier())
SFM_selected = selectorSFM.fit_transform(X, y)
column_names = X.columns[selectorSFM.get_support()]
pd.DataFrame(SFM_selected, columns = column_names).columns

Index(['Page', 'Likert_value', 'time', 'duration', 'distance',
       'x_axis_distance', 'y_axis_distance', 'real_ideal_trajectory_diff',
       'max_deviation', 'velocity', 'x_axis_vel', 'y_axis_vel', 'acc',
       'x_axis_acc', 'y_axis_acc', 'auc_diff', 'init_time', 'react_time'],
      dtype='object')

In [36]:
def orderLists(X, list_of_list):
    x_columns = X.columns
    x_weights = [0 for i in range(len(x_columns))]
    for lst in list_of_list:
        for i in range(len(x_columns)):
            x_weights[i] += lst.index(x_columns[i])         
    map_of_cols = [q for _, q in sorted(zip(x_weights, x_columns))]
    return map_of_cols    

In [37]:
list_of_list = [MI_column_order, Chi2_column_order, Fv_column_order, PrcCH_column_order, PrcFCL_column_order, PrcFRE_column_order]
ordered_list = orderLists(X, list_of_list)
ordered_list

['Likert_value',
 'auc_diff',
 'x_axis_vel',
 'max_deviation',
 'Big5',
 'velocity',
 'y_flips',
 'distance',
 'x_axis_distance',
 'x_axis_acc',
 'time',
 'visits',
 'real_ideal_trajectory_diff',
 'clicks',
 'y_axis_distance',
 'y_axis_vel',
 'acc',
 'duration',
 'x_flips',
 'y_axis_acc',
 'scrolling',
 'Page',
 'init_time',
 'react_time']

In [38]:
ordered_list[:10]

['Likert_value',
 'auc_diff',
 'x_axis_vel',
 'max_deviation',
 'Big5',
 'velocity',
 'y_flips',
 'distance',
 'x_axis_distance',
 'x_axis_acc']

## Sessions with highest probability of being correctly classified as FG/ H - for visualisation purposes only

In [39]:
# select static testing list of session
df[df['indicator_fg'] == 1].sort_values('mean_bfi2_c', ascending=False)[['Session', 'indicator_fg', 'mean_bfi2_c']][:10]['Session']

100    DjjvnfxVVsm3GVM8654buk14e
5      DrTk41pNckxJYIdilpta9qJBT
106    TZcocph2t67jhKfcZSdo0BBaU
103    rBwaeMLAX9nPiQy9BieuiFiAO
14     xEoYPtO0Mbwjg5PkET57lc4HB
40     JJYXFCVzYwSzr21WonkAkHugU
104    J7ZPIoCZ5AzpkwXK3jGK9XIxj
110    QJpzM76mOzzx41jlZ2cD68ZVr
60     IrbsxXnBAQgEce02vsUcX3uQR
70     niEXFrlkmqE1vexhOLRNtF7uY
Name: Session, dtype: object

In [40]:
df[df['indicator_fg'] == 0].sort_values('mean_bfi2_c', ascending=True)[['Session', 'indicator_fg', 'mean_bfi2_c']][:10]['Session']

65    7KpRcPUQDaJx1oROPQwCqSxxC
79    tnoOABvKMMfs9H3RJTq9kRkjN
31    zjMG8tuXTouFw6tDCP7oOVHSh
97    XHhPFBsnlOXgdBESumLQeLMQs
21    X3UTDLu8C6PzOEvMgh07iezGe
61    CphFMw9Nk0zqraysfoJtupGTQ
94    KDWak467FmFZN321qc8Qumwhc
66    lFVYq3ylPtO7eLfQl70dg1Vik
63    n0OYGf6ieNDHT94qgjog5peBJ
47    C4xb7O556Q4LRmp4p13qI6fbD
Name: Session, dtype: object